In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForMaskedLM,XLMRobertaForSequenceClassification

In [2]:
model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base",num_labels=5)
# Define device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import pandas as pd
train_url = '/kaggle/input/80-20ratiofinal/train8020.csv'
test_url = '/kaggle/input/80-20ratiofinal/test8020.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx',index_col=False)
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

import numpy as np
allcats = set(df_train['Category'].dropna().tolist())
allcats.add('Correct')
labeldict = {}
counter = 0
for i in allcats:
    labeldict[i] = counter
    counter += 1
labeldict

def manage(x):
    if x in labeldict:
        return labeldict[x]
    else:
        return labeldict['Correct']
df_train['Category'] = df_train['Category'].apply(lambda x:manage(x))
df_test['Category'] = df_test['Category'].apply(lambda x:manage(x))

data_no = 5

# Prepare the training data
train_texts = df_train['Comment'].tolist()
train_labels = df_train['Category'].tolist()

test_texts = df_test['Comment'].tolist()
test_labels = df_test['Category'].tolist()

In [6]:
# Tokenize and encode the training texts
train_encodings = tokenizer(train_texts, truncation=True, max_length=128,
        padding='max_length', return_tensors = 'pt')

# Convert the labels to tensors
train_labels = torch.tensor(train_labels)

# Create a PyTorch dataset
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'],
                                               train_encodings['attention_mask'],
                                               train_labels)

# Create a data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

model = model.to(device)

In [ ]:
from tqdm.notebook import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Set the model to training mode
model.train()

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

losses = []
accuracies = []  # To store accuracy per epoch
num_epochs = 5
# Training loop
for epoch in tqdm(range(num_epochs)):  # Number of training epochs
    running_loss = 0.0
    predicted_labels = []  # To store predicted labels for accuracy calculation
    true_labels = []  # To store true labels for accuracy calculation

    for batch in tqdm(train_loader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Convert logits to predicted labels
        _, predicted = torch.max(logits, dim=1)
        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(labels.cpu().tolist())

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

    # Calculate and store accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    accuracies.append(accuracy)

    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {accuracy:.4f}')

# Save the model
torch.save(model.state_dict(), 'model.pth')


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1256 [00:00<?, ?it/s]

In [11]:
#dgfdgdfgdgffdgdfd1212jhkhk

In [12]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def predict_labels(text):
    train_encodings = tokenizer(text, truncation=True, max_length=128,
        padding='max_length', return_tensors = 'pt')
    input_ids = train_encodings['input_ids'].to(device)
    attention_mask = train_encodings['attention_mask'].to(device)

    # Set the model to evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    

    return predicted_class.item(), probabilities[:,1].item()

In [13]:
predicted_labels = []
predicted_probs = []
for text in tqdm(test_texts):
    predicted_label, prob = predict_labels(text)
    predicted_labels.append(predicted_label)
    predicted_probs.append(prob)

# Calculate accuracy and F1 score
accuracy = accuracy_score(test_labels, predicted_labels)
# f1 = f1_score(test_labels, predicted_labels)
# roc_auc = roc_auc_score(test_labels, predicted_probs)

print('Accuracy:', accuracy)
# print('F1 Score:', f1)
# print('ROC-AUC:', roc_auc)

  0%|          | 0/2010 [00:00<?, ?it/s]

Accuracy: 0.7


In [14]:
print('Accuracy:', accuracy)

Accuracy: 0.7


In [15]:
from sklearn.metrics import roc_auc_score, classification_report

print('\nThe Classification Report is as follows\n')
print(classification_report(test_labels, predicted_labels, digits = 4))


The Classification Report is as follows

              precision    recall  f1-score   support

           0     0.7631    0.8323    0.7962      1157
           1     0.0000    0.0000    0.0000       128
           2     0.2727    0.0435    0.0750        69
           3     0.5934    0.6827    0.6349       498
           4     0.6159    0.6392    0.6273       158

    accuracy                         0.7000      2010
   macro avg     0.4490    0.4396    0.4267      2010
weighted avg     0.6440    0.7000    0.6675      2010



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
